# <center> Hyperparameter Tuning for Generative Adversarial Network (NLS-KDD) <center/>

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

import utils , preprocessing , cgan , VannilaGan, wgan 
from matplotlib import pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from scipy.stats import norm
from tensorflow.keras.losses import kullback_leibler_divergence

Using TensorFlow backend.


## Read Data

In [2]:
train,test, label_mapping = preprocessing.get_data(encoding="Label")

data_cols = list(train.columns[ train.columns != 'label' ])
x_train , x_test = preprocessing.preprocess(train,test,data_cols,"Robust",True)

y_train = x_train.label.values
y_test = x_test.label.values

data_cols = list(x_train.columns[ x_train.columns != 'label' ])

In [ ]:
to_drop = preprocessing.get_contant_featues(x_train,data_cols)
x_train.drop(to_drop, axis=1,inplace=True)
x_test.drop(to_drop, axis=1,inplace=True)

data_cols = list(x_train.columns[ x_train.columns != 'label' ])

## Set Hyperparams

In [3]:
rand_dim = 32
base_n_count = 100

combined_ep = 2000
batch_size = 128 if len(x) > 128 else len(x)

ep_d = 1
ep_g = 2
learning_rate = 0.0001

## Generator & Discriminator Definitions

In [4]:
def get_discrimnator(data_dim, min_num_neurones,n_layers, lr ,optimizer ,activation_f):
    model = tf.keras.models.Sequential(name='Discriminator')
    
    model.add(Dense(min_num_neurones*n_layers, activation=activation_f,input_dim = data_dim))
    for n in reversed(range(1,n_layers)):
        model.add(Dense(n*min_num_neurones, activation = activation_f))
        
    model.add(Dense(1,activation="sigmoid"))
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    K.set_value(model.optimizer.lr, lr)
    
    return model

def get_generator(data_dim, min_num_neurones,noise_dim,n_layers, lr ,optimizer ,activation_f):
    model = tf.keras.models.Sequential(name='Generator')
    
    model.add(Dense(min_num_neurones, activation=activation_f,input_dim = noise_dim ))
    for n in range(1,n_layers):
        model.add(Dense(n*min_num_neurones, activation = activation_f))
        
    model.add(Dense(data_dim,activation=activation_f))
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    K.set_value(model.optimizer.lr, lr)
    
    return model

def create_gan(discriminator, generator, z_dim):
    discriminator.trainable=False
    gan_input = Input(shape=(z_dim,))
    
    gan_output = discriminator(generator(gan_input))
    gan= Model(inputs = gan_input, outputs = gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='sgd')
    return gan

In [5]:
def get_batch(X, batch_size=1):
    batch_ix = np.random.choice(len(X), batch_size, replace=False)
    return X[batch_ix]

In [6]:
def training1(arguments,X):
    K.clear_session()
    
    [rand_noise_dim, nb_steps, batch_size,D_epochs, G_epochs, min_num_neurones, lr, n_layers ,optimizer ,activation_f ] = arguments
    
    data_dim = X.shape[1]
    combined_loss, disc_loss_generated, disc_loss_real = [], [], []
    
    # Creating GAN
    generator = get_generator(data_dim, min_num_neurones,rand_noise_dim,n_layers, lr ,optimizer ,activation_f)
    discriminator = get_discrimnator(data_dim, min_num_neurones,n_layers, lr ,optimizer ,activation_f)
    adversarial_model = create_gan(discriminator, generator,rand_noise_dim)
    
    print(generator.summary())
    print(discriminator.summary())
    print(adversarial_model.summary())
    
    #print("batch_size : ", batch_size)
    #print("rand_dim : ",rand_dim)
    #Start training
    for epoch in range(1,nb_steps + 1 ):
        K.set_learning_phase(1)
        
        #Train Discriminator
        discriminator.trainable=True
        for i in range(D_epochs):
            np.random.seed(i+epoch)
        
            noise = np.random.normal(0,1, size=(batch_size, rand_noise_dim))
            generated_samples = generator.predict(noise)
            real_samples = get_batch(X,batch_size)
            
            d_l_r = discriminator.train_on_batch(real_samples, np.random.uniform(low=0.999, high=1.0, size=batch_size))
            d_l_g = discriminator.train_on_batch(generated_samples, np.random.uniform(low=0.0, high=0.0001, size=batch_size))
        
        #Freeze Discriminator
        #print("D lr : {}".format(discriminator.optimizer.get_config()['lr']))
        discriminator.trainable = False
        disc_loss_generated.append(d_l_g)
        disc_loss_real.append(d_l_r)
        
        #Train Generator
        for i in range(G_epochs):
            np.random.seed(i+epoch)
            
            noise = np.random.normal(0,1, size = (batch_size, rand_noise_dim))
            loss = adversarial_model.train_on_batch(noise, np.random.uniform(low=0.999, high=1.0, size=batch_size))
            
        combined_loss.append(loss)
        
        #Do checkpointing
        if epoch % 10 == 0:
            K.set_learning_phase(0)
            test_size = len(X)
            
            z = np.random.normal(3,2,size=(test_size, rand_noise_dim))

            fake_pred = np.array(adversarial_model.predict(z)).ravel()
            real_pred = np.array(discriminator.predict(X)).ravel()

            modelAccuracy(fake_pred,real_pred)
        
    return dict({"generator_model":generator,"discriminator_model":discriminator,\
            "combined_model":adversarial_model,"generator_loss":combined_loss,\
            "disc_loss_generated":disc_loss_generated,"disc_loss_real": disc_loss_real})

In [7]:
att_ind = np.where(train.label == label_mapping["probe"])[0]

x = x_train[att_ind]

# rand_dim = 32
# base_n_count = 100
# batch_size = 128 if len(x) > 128 else len(x)
# combined_ep = 100
# learning_rate = 0.0001

rand_dim = np.arange(10,110,10)
base_n_count = np.arange(3,41,3)
ephocs = np.arange(100,5000,100)
batch_sizes = [64,128,250,300,350]
learning_rates = np.logspace(-1,-4,num=10)

optimizers = ["sgd", "RMSprop", "adam", "Adagrad", "Adamax","Nadam"]
activation_func = ["tanh","relu","softplus","linear","elu"]

ep_d = 1
ep_g = 1

## Test

In [11]:
from collections import defaultdict

In [8]:
k = []
for ep in ephocs:
    for lr in learning_rates:
        for op in optimizers:
            for ac in activation_func:
                arguments = [32, ep, 300,ep_d, ep_g, 50,lr, 4 ,op ,ac]
                res = training1(arguments,x)
                print("Saving results to file....")
                comb_model = res["combined_model"]
                comb_model.save(f'model{ep}_{lr}_{op}_{ac}.h5')
                k.append(res)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                1650      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_3 (Dense)              (None, 150)               15150     
_________________________________________________________________
dense_4 (Dense)              (None, 41)                6191      
Total params: 30,641
Trainable params: 30,641
Non-trainable params: 0
_________________________________________________________________
None
_____________________________________________________________

Discriminator accuracy on Fake : 0.0, Real : 1.0
Discriminator accuracy on Fake : 0.0, Real : 1.0
Discriminator accuracy on Fake : 0.0, Real : 1.0
finished 0ne
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                1650      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_3 (Dense)              (None, 150)               15150     
_________________________________________________________________
dense_4 (Dense)              (None, 41)                6191      
Total params: 30,641
Trainable params: 30,641
Non-trainable params: 0
____________________________________________

Discriminator accuracy on Fake : 0.6004632807137955, Real : 0.47777968428277284
Discriminator accuracy on Fake : 0.5339739190116678, Real : 0.9572752230610844
Discriminator accuracy on Fake : 0.4889327385037749, Real : 0.9936513383665065
finished 0ne
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                1650      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_3 (Dense)              (None, 150)               15150     
_________________________________________________________________
dense_4 (Dense)              (None, 41)                6191      
Total params: 30,641
Tr

Discriminator accuracy on Fake : 1.0, Real : 0.0
Discriminator accuracy on Fake : 1.0, Real : 0.2676733013040494
Discriminator accuracy on Fake : 1.0, Real : 0.5106382978723404
Discriminator accuracy on Fake : 1.0, Real : 0.6494509265614276
Discriminator accuracy on Fake : 1.0, Real : 0.6920041180507893
finished 0ne
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                1650      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_3 (Dense)              (None, 150)               15150     
_________________________________________________________________
dense_4 (Dense)       

Discriminator accuracy on Fake : 0.0835621139327385, Real : 0.9957103637611531
Discriminator accuracy on Fake : 0.983270418668497, Real : 0.9274193548387096
Discriminator accuracy on Fake : 0.5879375428963624, Real : 0.9275909402882636
Discriminator accuracy on Fake : 0.04649965682910089, Real : 0.9981125600549073
Discriminator accuracy on Fake : 0.002916952642415923, Real : 0.9961393273850377
finished 0ne
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                1650      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_3 (Dense)              (None, 150)               15150  

Discriminator accuracy on Fake : 1.0, Real : 0.950754975978037
Discriminator accuracy on Fake : 1.0, Real : 0.9491249142072752
Discriminator accuracy on Fake : 1.0, Real : 0.8940459849004805
Discriminator accuracy on Fake : 1.0, Real : 0.9463795470144132
Discriminator accuracy on Fake : 1.0, Real : 0.9405456417295813
finished 0ne
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                1650      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_3 (Dense)              (None, 150)               15150     
_________________________________________________________________
dense_4 